In [1]:
import tensorflow as tf
from nilearn import connectome
import pickle
import numpy as np

In [2]:
with open("time_series.pkl", "rb") as f:
    time_series_data = pickle.load(f)

In [8]:
FCs = {} # 所有数据计算得到的功能连接
for sub, ses in time_series_data.items():
    FCs[sub] = {}
    for op, runs in ses.items():
        FCs[sub][op] = {}
        for run, items in runs.items():
            if "time_series" in items:
                FCs[sub][op][run] = connectome.ConnectivityMeasure(kind="correlation").fit_transform([items["time_series"]])[0]

In [9]:
with open("FCs.pkl", "wb") as f:
    pickle.dump(FCs, f)

In [2]:
with open("FCs.pkl", "rb") as f:
    FCs = pickle.load(f)

In [3]:
data = []
labels = []
for sub, ses in FCs.items():
    for op, runs in ses.items():
        for run, items in runs.items():
            data.append(items)
            labels.append(op)

In [4]:
np.savez("es_dataset", data=data, labels=labels)

In [5]:
with np.load("es_dataset.npz") as dataset:
    data = dataset["data"]
    labels = dataset["labels"]

In [6]:
es_dataset = tf.data.Dataset.from_tensor_slices((data, labels))